## Full Run

In [1]:
import os
os.chdir('..')

In [2]:
Xtrain_dir = 'solar/data/kaggle_solar/train/'
Xtest_dir = 'solar/data/kaggle_solar/test'
ytrain_file = 'solar/data/kaggle_solar/train.csv'
station_file = 'solar/data/kaggle_solar/station_info.csv'
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model
import solar.report.submission
import numpy as np

In [14]:
# Choose up to 98 stations; not specifying a station means to use all that fall within the given lats and longs. If the
# parameter 'all' is given, then it will use all stations no matter the provided lats and longs
station = ['all']

# Determine which dates will be used to train the model. No specified date means use the entire set from 1994-01-01 
# until 2007-12-31.
train_dates = ['1994-01-01', '2007-12-31']

#2008-01-01 until 2012-11-30
test_dates = ['2008-01-01', '2012-11-30']

station_layout = True

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model = [0]

# Aggregate over all times
times = ['all']

default_grid = {'type':'relative', 'axes':{'var':var, 'models':model, 'times':times,
                                          'station':station}}
# This just uses the station_names as another feature
stat_names = {'type':'station_names'}

frac_dist = {'type':'frac_dist'}

days_solstice = {'type':'days_from_solstice'}
days_cold = {'type':'days_from_coldest'}

all_feats = [stat_names, default_grid, frac_dist, days_solstice, days_cold]
#all_feats = [stat_names, days_solstice, days_cold]

In [4]:
import solar.report.submission
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model

In [15]:
# test combination of station names and grid
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

external = True

input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, all_feats, external)

INFO:solar.wrangle.wrangle:Started building test and training data
INFO:solar.wrangle.wrangle:Features: [{'type': 'station_names'}, {'axes': {'var': ['all'], 'models': [0], 'station': ['all'], 'times': ['all']}, 'type': 'relative'}, {'type': 'frac_dist'}, {'type': 'days_from_solstice'}, {'type': 'days_from_coldest'}]
INFO:solar.wrangle.wrangle:Train dates: ['1994-01-01', '2007-12-31']
INFO:solar.wrangle.wrangle:Test dates: ['2008-01-01', '2012-11-30']
INFO:solar.wrangle.wrangle:Stations: ['all']
INFO:solar.wrangle.wrangle:Finished building test and training data


In [16]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'

model = Model.model(input_data, Ridge, {'alpha':np.logspace(-6,5,10,base=10)}, 10, 
                    error_formula, normalize=True)

INFO:solar.analyze.model:Started building model
INFO:solar.analyze.model:Train input columns: Index([                             u'stat_ACME',
                                    u'stat_ADAX',
                                    u'stat_ALTU',
                                    u'stat_APAC',
                                    u'stat_ARNE',
                                    u'stat_BEAV',
                                    u'stat_BESS',
                                    u'stat_BIXB',
                                    u'stat_BLAC',
                                    u'stat_BOIS',
       ...
          (u'relative', 0, u'tmp_2m', u'NE', 24),
          (u'relative', 0, u'tmp_2m', u'NW', 24),
         (u'relative', 0, u'tmp_sfc', u'SE', 24),
         (u'relative', 0, u'tmp_sfc', u'SW', 24),
         (u'relative', 0, u'tmp_sfc', u'NE', 24),
         (u'relative', 0, u'tmp_sfc', u'NW', 24),
                     (u'frac_dist', u'lat_dist'),
                    (u'frac_dist', u'long_dis

KeyboardInterrupt: 

In [7]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics

error_formula = 'mean_absolute_error'

model = Model.model_from_pickle('input_2016-02-06-18-17-28.p', GradientBoostingRegressor, {'n_estimators':range(100,500, 100), 'learning_rate':np.logspace(-3,1,5,base=10)}, 10, 
                    error_formula, loss='ls', max_depth=1, random_state=0)

INFO:solar.analyze.model:Started building model
INFO:solar.analyze.model:Train input columns: Index([                             u'stat_ACME',
                                    u'stat_ADAX',
                                    u'stat_ALTU',
                                    u'stat_APAC',
                                    u'stat_ARNE',
                                    u'stat_BEAV',
                                    u'stat_BESS',
                                    u'stat_BIXB',
                                    u'stat_BLAC',
                                    u'stat_BOIS',
       ...
       (u'relative', 0, u'uswrf_sfc', u'NE', 24),
       (u'relative', 0, u'uswrf_sfc', u'NW', 24),
       (u'relative', 0, u'dswrf_sfc', u'SE', 24),
       (u'relative', 0, u'dswrf_sfc', u'SW', 24),
       (u'relative', 0, u'dswrf_sfc', u'NE', 24),
       (u'relative', 0, u'dswrf_sfc', u'NW', 24),
                     (u'frac_dist', u'lat_dist'),
                    (u'frac_dist', u'long_dis

KeyboardInterrupt: 

In [4]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.submit_from_pickle('model_2016-02-06-18-21-41.p', 'input_2016-02-06-18-17-28.p', True)



INFO:solar.report.submission:Started building submission file
INFO:solar.report.submission:Finished building submission file


IOError: [Errno 2] No such file or directory: 'solar/data/kaggle_solar/submissions/submit_2016-02-06-19-04-48.p'